## Sissejuhatus

See õppetund käsitleb:
- Mis on funktsiooni kutsumine ja selle kasutusjuhtumid
- Kuidas luua funktsiooni kutse, kasutades OpenAI-d
- Kuidas integreerida funktsiooni kutse rakendusse

## Õpieesmärgid

Pärast selle õppetunni läbimist tead ja mõistad, kuidas:

- Funktsiooni kutsumise eesmärk
- Seadistada funktsiooni kutse, kasutades OpenAI teenust
- Kujundada tõhusaid funktsiooni kutseid oma rakenduse kasutusjuhtumi jaoks


## Funktsioonikõnede mõistmine

Selleks õppetükiks tahame ehitada oma haridusettevõttele funktsiooni, mis võimaldab kasutajatel kasutada vestlusrobotit tehniliste kursuste leidmiseks. Soovitame kursusi, mis sobivad nende oskustasemega, praeguse ametikoha ja huvipakkuva tehnoloogiaga.

Selle lõpuleviimiseks kasutame kombinatsiooni:
 - `OpenAI`, et luua kasutajale vestluskogemus
 - `Microsoft Learn Catalog API`, et aidata kasutajatel leida kursusi vastavalt kasutaja päringule
 - `Function Calling`, et võtta kasutaja päring ja saata see funktsioonile API-päringu tegemiseks.

Alustamiseks vaatame, miks me üldse tahaksime kasutada funktsioonikõnet:

print("Sõnumid järgmises päringus:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # saada GPT-lt uus vastus, kus ta näeb funktsiooni vastust


print(second_response.choices[0].message)


### Miks kasutada funktsioonikõnet

Kui olete selle kursuse mõne teise õppetüki lõpetanud, mõistate tõenäoliselt suurte keelemudelite (LLM-ide) kasutamise võimsust. Loodetavasti näete ka mõningaid nende piiranguid.

Funktsioonikõne on OpenAI teenuse funktsioon, mis on loodud järgmiste väljakutsete lahendamiseks:

Ebaühtlane vastuse vormindus:
- Enne funktsioonikõnet olid suurte keelemudelite vastused struktureerimata ja ebajärjekindlad. Arendajad pidid kirjutama keerulist valideerimiskoodi, et käsitleda iga väljundi variatsiooni.

Piiratud integratsioon väliste andmetega:
- Selle funktsiooni eel oli keeruline kaasata andmeid rakenduse teistest osadest vestluse konteksti.

Vastuse vormingute standardiseerimise ja sujuva integratsiooni võimaldamisega väliste andmetega lihtsustab funktsioonikõne arendust ning vähendab täiendava valideerimisloogika vajadust.

Kasutajad ei saanud vastuseid küsimustele nagu "Milline on hetkeilm Stockholmis?". See on tingitud sellest, et mudelid olid piiratud andmete treenimise ajaga.

Vaatame allolevat näidet, mis illustreerib seda probleemi:

Oletame, et soovime luua õpilaste andmebaasi, et saaksime neile soovitada sobiva kursuse. Allpool on kaks õpilaste kirjeldust, mis on andmete poolest väga sarnased.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Soovime seda saata LLM-ile, et andmeid töödelda. Seda saab hiljem meie rakenduses kasutada API-le saatmiseks või andmebaasi salvestamiseks.

Loome kaks identselt prompti, milles juhendame LLM-i, millist teavet me otsime:


Me tahame seda saata LLM-ile, et see analüüsiks meie toote jaoks olulisi osi. Nii saame luua kaks identselt sama juhist sisaldavat prompti LLM-ile:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Pärast nende kahe prompti loomist saadame need LLM-ile, kasutades `openai.ChatCompletion`. Me salvestame prompti muutujasse `messages` ja määrame rolli `user`. See on selleks, et jäljendada kasutaja sõnumi kirjutamist vestlusrobotile.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Nüüd saame mõlemad päringud saata LLM-ile ja uurida saadud vastust.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Kuigi promptid on samad ja kirjeldused sarnased, võime saada erinevaid vorminguid omaduse `Grades` jaoks.

Kui käivitada ülaltoodud lahter mitu korda, võib vorming olla kas `3.7` või `3.7 GPA`.

See on tingitud sellest, et LLM võtab vastu struktureerimata andmeid kirjutatud prompti kujul ja tagastab samuti struktureerimata andmeid. Me vajame struktureeritud vormingut, et teada, mida oodata, kui neid andmeid salvestada või kasutada.

Funktsionaalse kutsumise abil saame tagada, et saame tagasi struktureeritud andmed. Funktsionaalse kutsumise kasutamisel LLM tegelikult ei kutsu ega käivita ühtegi funktsiooni. Selle asemel loome struktuuri, mida LLM peab oma vastustes järgima. Seejärel kasutame neid struktureeritud vastuseid, et teada, millist funktsiooni meie rakendustes käivitada.


![Funktsiooni Kutsumise Vooskeem](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.et.png)


Seejärel saame funktsioonist tagastatu võtta ja saata selle tagasi LLM-ile. LLM vastab seejärel kasutades loomulikku keelt, et vastata kasutaja päringule.


### Funktsioonikõnede kasutamise juhtumid

**Väliste tööriistade kutsumine**  
Vestlusrobotid on suurepärased kasutajate küsimustele vastuste andmisel. Funktsioonikõnede abil saavad vestlusrobotid kasutada kasutajate sõnumeid teatud ülesannete täitmiseks. Näiteks võib üliõpilane paluda vestlusrobotil „Saada e-kiri minu juhendajale, öeldes, et vajan selle teema kohta rohkem abi“. See võib teha funktsioonikõne `send_email(to: string, body: string)`

**API või andmebaasi päringute loomine**  
Kasutajad saavad leida teavet loomulikus keeles, mis teisendatakse vormindatud päringuks või API-päringuks. Näiteks võib õpetaja küsida „Kes on õpilased, kes lõpetasid viimase ülesande“, mis võib kutsuda funktsiooni nimega `get_completed(student_name: string, assignment: int, current_status: string)`

**Struktureeritud andmete loomine**  
Kasutajad saavad võtta tekstiploki või CSV-faili ja kasutada LLM-i, et sellest olulist teavet välja võtta. Näiteks võib üliõpilane teisendada Vikipeedia artikli rahulepingute kohta, et luua tehisintellekti mälukaardid. Seda saab teha funktsiooniga `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Esimese funktsiooni väljakutse loomine

Funktsiooni väljakutse loomise protsess hõlmab 3 peamist sammu:
1. Chat Completions API kutsumine koos teie funktsioonide nimekirja ja kasutaja sõnumiga
2. Mudeli vastuse lugemine, et sooritada tegevus, nt funktsiooni või API kõne täitmine
3. Teise kõne tegemine Chat Completions API-le koos teie funktsiooni vastusega, et kasutada seda teavet kasutajale vastuse loomiseks.


![Funktsiooni Kutsumise Voog](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.et.png)


### Funktsiooni väljakutse elemendid

#### Kasutaja sisend

Esimene samm on luua kasutaja sõnum. Seda saab dünaamiliselt määrata, võttes väärtuse tekstisisendist või võite siin väärtuse määrata. Kui see on teie esimene kord töötada Chat Completions API-ga, peame määratlema sõnumi `role` ja `content`.

`role` võib olla kas `system` (reeglite loomine), `assistant` (mudel) või `user` (lõppkasutaja). Funktsiooni väljakutse puhul määrame selle kui `user` ja näidisülesande.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Funktsioonide loomine.

Järgmisena määratleme funktsiooni ja selle funktsiooni parameetrid. Siin kasutame ainult ühte funktsiooni nimega `search_courses`, kuid võite luua mitu funktsiooni.

**Oluline**: Funktsioonid on kaasatud süsteemiteatesse LLM-ile ja need arvestatakse teie kasutatavate saadaolevate tokenite hulka.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Määratlused** 

Funktsiooni definitsiooni struktuur koosneb mitmest tasemest, millest igaühel on omadused. Siin on ülevaade pesastatud struktuurist:

**Kõrgeima taseme funktsiooni omadused:**

`name` - Funktsiooni nimi, mida soovime kutsuda. 

`description` - Kirjeldus, kuidas funktsioon töötab. Siin on oluline olla konkreetne ja selge. 

`parameters` - Loend väärtustest ja vormingust, mida soovite, et mudel oma vastuses toodaks. 

**Parameetrite objekti omadused:**

`type` - Parameetrite objekti andmetüüp (tavaliselt "object")

`properties` - Loend konkreetsetest väärtustest, mida mudel kasutab oma vastuses. 

**Individuaalsete parameetrite omadused:**

`name` - Implitsiitselt määratud omaduse võtme järgi (nt "role", "product", "level")

`type` - Selle konkreetse parameetri andmetüüp (nt "string", "number", "boolean") 

`description` - Konkreetse parameetri kirjeldus 

**Valikulised omadused:**

`required` - Massiiv, mis loetleb, millised parameetrid on funktsiooni kutsumiseks kohustuslikud.


### Funktsiooni kutsumine  
Pärast funktsiooni defineerimist peame nüüd selle lisama Chat Completion API kutsesse. Teeme seda, lisades päringule `functions`. Antud juhul on see `functions=functions`.  

Samuti on võimalik seada `function_call` väärtuseks `auto`. See tähendab, et laseme LLM-il otsustada, millist funktsiooni tuleks kasutaja sõnumi põhjal kutsuda, selle asemel et määrata see ise.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Nüüd vaatame vastust ja näeme, kuidas see on vormindatud:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Näete, et funktsiooni nimi kutsutakse välja ja kasutaja sõnumist suutis LLM leida andmed, mis sobivad funktsiooni argumentidega.


## 3.Funktsioonikõnede integreerimine rakendusse. 


Pärast seda, kui oleme testinud LLM-i vormindatud vastust, saame selle nüüd rakendusse integreerida. 

### Voogude haldamine 

Selle rakendusse integreerimiseks teeme järgmised sammud: 

Esiteks teeme kõne OpenAI teenustele ja salvestame sõnumi muutujasse nimega `response_message`. 


In [ ]:
response_message = response.choices[0].message

Nüüd määratleme funktsiooni, mis kutsub Microsoft Learn API-d, et saada kursuste nimekiri:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Parima tavana kohaselt vaatame seejärel, kas mudel soovib funktsiooni kutsuda. Pärast seda loome ühe saadaolevatest funktsioonidest ja sobitame selle kutsutava funktsiooniga.  
Seejärel võtame funktsiooni argumendid ja kaardistame need LLM-i argumentidele.

Lõpuks lisame funktsiooni kutsumise sõnumi ja väärtused, mis tagastati `search_courses` sõnumi poolt. See annab LLM-ile kogu vajaliku teabe, et vastata kasutajale loomulikus keeles.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Nüüd saadame uuendatud sõnumi LLM-ile, et saaksime loomuliku keele vastuse API JSON-vormingus vastuse asemel.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Koodi väljakutse

Suurepärane töö! OpenAI funktsioonikõnede õppimise jätkamiseks võid luua: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Funktsiooni rohkem parameetreid, mis võivad aidata õppijatel leida rohkem kursusi. Saad saadaolevaid API parameetreid vaadata siin:  
 - Loo teine funktsioonikõne, mis võtab õppijalt rohkem teavet, näiteks nende emakeele  
 - Loo veahaldus juhuks, kui funktsioonikõne ja/või API kõne ei tagasta sobivaid kursusi


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Vastutusest loobumine**:
See dokument on tõlgitud kasutades tehisintellektil põhinevat tõlketeenust [Co-op Translator](https://github.com/Azure/co-op-translator). Kuigi püüame tagada täpsust, palun arvestage, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Originaaldokument selle emakeeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitatakse kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti mõistmiste eest.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
